# R: DoubleML for Difference-in-Differences

In this example, we demonstrate, how `DoubleML` can be used in combination with the [`did` package for R](https://bcallaway11.github.io/did/index.html) in order to estimate group-time average treatment effects in difference-in-difference (DiD) models with multiple periods.

In [ ]:
library(DoubleML)
library(did)
library(mlr3)
library(mlr3learners)

# suppress messages during fitting
lgr::get_logger("mlr3")$set_threshold("warn")

set.seed(1234)


## Demo example from `did`

We will demonstrate the use of `DoubleML` for DiD in the [introductory example](https://bcallaway11.github.io/did/articles/did-basics.html) of the `did` package. 

In [ ]:
# Generate data, original code available at https://github.com/bcallaway11/did/blob/master/vignettes/did-basics.Rmd
time.periods <- 4
sp <- reset.sim()
sp$te <- 0

set.seed(1814)

# generate dataset with 4 time periods
time.periods <- 4

# add dynamic effects
sp$te.e <- 1:time.periods

# generate data set with these parameters
# here, we dropped all units who are treated in time period 1 as they do not help us recover ATT(g,t)'s.
dta <- build_sim_dataset(sp)

# How many observations remained after dropping the ``always-treated'' units
nrow(dta)
#This is what the data looks like
head(dta)

In [ ]:
n <- 10000
decision_effect <- -2
instrument_effect <- 0.7

confounder <- rbinom(n, 1, 0.3)
instrument <- rbinom(n, 1, 0.5)
decision <- as.numeric(runif(n) <= instrument_effect*instrument + 0.4*confounder)
outcome <- 30 + decision_effect*decision + 10 * confounder + rnorm(n, sd=2)
df <- data.frame(instrument, decision, outcome)

### Comparison to `did` package

By default, estimation in `did` is based on (unpenalized) linear and logistic regression. Let's start with this default model first.

In [ ]:
# estimate group-time average treatment effects using att_gt method
example_attgt <- att_gt(yname = "Y",
                        tname = "period",
                        idname = "id",
                        gname = "G",
                        xformla = ~X,
                        data = dta
                        )

# summarize the results
summary(example_attgt)

### Using ML for DiD: Integrating `DoubleML` in `did`

As described in our [Section on DiD models in the user guide](https://docs.doubleml.org/stable/guide/models.html#difference-in-differences-models-did), [Sant'Anna and Zhao (2020)](https://linkinghub.elsevier.com/retrieve/pii/S0304407620301901) have developed a doubly robust DiD model which is compatible with ML-based estimation. As this doubly robust model is internally used in `did`, it is possible to use `DoubleML` here to obtain valid point estimates and confidence intervals. For this, we need to write a wrapper around a `DoubleMLIRM` model and pass it to `did` as a custom estimation approach. Once this is implemented, we can use all the nice features and advantages of the `did` package.

For now, let's abstract from using fancy ML algorithms to keep the comparison to the classic `did` implementation simple. Hence, we will use linear and logistic regression for the nuisance compontents in the DiD model.

In [ ]:
# DoubleML wrapper for did
set.seed(1234)
doubleml_did_linear <- function(y1, y0, D, covariates,
                         ml_g = lrn("regr.lm"),
                         ml_m = lrn("classif.log_reg"),
                         n_folds = 10, n_rep = 1, ...) {
  
  # warning if n_rep > 1 to handle mapping from psi to inf.func
  if (n_rep > 1) {
    warning("n_rep > 1 is not supported.")
  }
  # Compute difference in outcomes
  delta_y <- y1 - y0
  # Prepare data backend
  dml_data = DoubleML::double_ml_data_from_matrix(X = covariates, y = delta_y, d = D)
  # Compute the ATT
  dml_obj = DoubleML::DoubleMLIRM$new(dml_data, ml_g = ml_g, ml_m = ml_m, score = "ATTE", n_folds = n_folds)
  dml_obj$fit()
  att = dml_obj$coef[1]
  # Return results
  inf.func <- dml_obj$psi[, 1, 1]
  output <- list(ATT = att, att.inf.func = inf.func)
  return(output)
}

example_attgt_dml_linear <- att_gt(yname = "Y",
                        tname = "period",
                        idname = "id",
                        gname = "G",
                        xformla = ~X,
                        data = dta,
                        est_method = doubleml_did_linear)


summary(example_attgt_dml_linear)

Any differences from the default `did` implementation arise due to sampling randomness, because `DoubleML` uses cross-fitting internally, which is not necessary if classical parametric estimation methods are used.

Next, let's demonstrate how we can use more complex ML learners. For this, we just have to pass another `mlr3` learner through the wrapper, for example a random forest. Please note that the original data generating process is linear, such that we don't expect random forest to lead to better results than the linear learners. We provide a variant of the wrapper that includes an evaluation of the nuisance predictions at the end of this notebook.

In [ ]:
# DoubleML wrapper for did with random forest learner
set.seed(1234)

doubleml_did_rf <- function(y1, y0, D, covariates,
  ml_g = lrn("regr.ranger"),
  ml_m = lrn("classif.ranger"),
  n_folds = 10, n_rep = 1, ...) {

# warning if n_rep > 1 to handle mapping from psi to inf.func
if (n_rep > 1) {
warning("n_rep > 1 is not supported.")
}
# Compute difference in outcomes
delta_y <- y1 - y0
# Prepare data backend
dml_data = DoubleML::double_ml_data_from_matrix(X = covariates, y = delta_y, d = D)
# Compute the ATT
dml_obj = DoubleML::DoubleMLIRM$new(dml_data, ml_g = ml_g, ml_m = ml_m, score = "ATTE", n_folds = n_folds)
dml_obj$fit()
att = dml_obj$coef[1]
# Return results
inf.func <- dml_obj$psi[, 1, 1]
output <- list(ATT = att, att.inf.func = inf.func)
return(output)
}

example_attgt_dml_rf <- att_gt(yname = "Y",
 tname = "period",
 idname = "id",
 gname = "G",
 xformla = ~X,
 data = dta,
 est_method = doubleml_did_rf)

summary(example_attgt_dml_rf)

We can see that the results are not dramatically different from the results before. We can observe from the larger standard errors that the default random forest learners seems to be a less precise prediction rule.

### Exploiting the Functionalities of `did`

The `did` package offers various tools for multi-period DiD models, for example plotting the group-time average treatment effects, which can be exploited just as in the native `did` usage.

In [ ]:
# Plot group-time average treatment effects
ggdid(example_attgt_dml_linear)

It's also possible to calculate aggregated effect estimates. Please note that, the results are again very close to those in [the original notebook](https://bcallaway11.github.io/did/articles/did-basics.html#aggregating-group-time-average-treatment-effects).

In [ ]:
agg.simple <- aggte(example_attgt_dml_linear, type = "simple")
summary(agg.simple)

### Details on Predictive Performance

We can add an evaluation functionality to the wrapper to assess how the predictive performance from the linear and logistic regression differ from that of the random forest learner.

In [ ]:
library(mlr3measures)

# Add a wrapper that computes the RMSE and accuracy for the nuisance components, can be customized by providing custom measures
eval_preds = function(y, d, predictions, params_names, custom_measures = NULL) {
  measures_res = list()

  if (!is.null(custom_measures)) {
    # Alternatively provide a named list with custom evaluation functions
    measure_funcs = list()
  } else {
    measure_funcs = list()
    measure_funcs[['ml_m']] = mlr3measures::acc
    measure_funcs[['ml_g0']] = mlr3measures::rmse
  }

  for (param_name in params_names) {
    preds = predictions[[param_name]][, 1, 1]

  if (param_name == "ml_m") {
    obs = d
    # map probability predictions to binary
    preds = as.factor(ifelse(preds > 0.5, 1, 0))
    obs = as.factor(preds)
  }

  else if (param_name == "ml_g0") {
    obs = y[d == 0]
    preds = preds[d == 0]
  }

  if (param_name == "ml_g1") {
    next
  }

  else {
    measure_func = measure_funcs[[param_name]]
    measure_pred = measure_func(obs, preds)

    measures_res[[param_name]] = measure_pred
  }

  }
    return(measures_res)
}

# evaluate learner performance: linear models
doubleml_did_eval_linear <- function(y1, y0, D, covariates,
  ml_g = lrn("regr.lm"),
  ml_m = lrn("classif.log_reg"),
  n_folds = 10, n_rep = 1, ...) {

  # warning if n_rep > 1 to handle mapping from psi to inf.func
  if (n_rep > 1) {
    warning("n_rep > 1 is not supported.")
  }
  # Compute difference in outcomes
  delta_y <- y1 - y0
  # Prepare data backend
  dml_data = DoubleML::double_ml_data_from_matrix(X = covariates, y = delta_y, d = D)
  # Compute the ATT
  dml_obj = DoubleML::DoubleMLIRM$new(dml_data, ml_g = ml_g, ml_m = ml_m, score = "ATTE", n_folds = n_folds)
  dml_obj$fit(store_predictions = TRUE)
  att = dml_obj$coef[1]
  # Return results
  inf.func <- dml_obj$psi[, 1, 1]

  # Evaluate learner performance
  predictions = dml_obj$predictions
  params_names = dml_obj$params_names()
  eval_predictions = eval_preds(delta_y, D, predictions, params_names)
  print(eval_predictions)

  output <- list(ATT = att, att.inf.func = inf.func)
  return(output)
  }

  library(mlr3measures)

# evaluate learner performance: random forest
doubleml_did_eval_rf <- function(y1, y0, D, covariates,
  ml_g = lrn("regr.ranger"),
  ml_m = lrn("classif.ranger"),
  n_folds = 10, n_rep = 1, ...) {

  # warning if n_rep > 1 to handle mapping from psi to inf.func
  if (n_rep > 1) {
    warning("n_rep > 1 is not supported.")
  }
  # Compute difference in outcomes
  delta_y <- y1 - y0
  # Prepare data backend
  dml_data = DoubleML::double_ml_data_from_matrix(X = covariates, y = delta_y, d = D)
  # Compute the ATT
  dml_obj = DoubleML::DoubleMLIRM$new(dml_data, ml_g = ml_g, ml_m = ml_m, score = "ATTE", n_folds = n_folds)
  dml_obj$fit(store_predictions = TRUE)
  att = dml_obj$coef[1]
  # Return results
  inf.func <- dml_obj$psi[, 1, 1]

  # Evaluate learner performance
  predictions = dml_obj$predictions
  params_names = dml_obj$params_names()
  eval_predictions = eval_preds(delta_y, D, predictions, params_names)
  print(eval_predictions)

  output <- list(ATT = att, att.inf.func = inf.func)
  return(output)
  }


Running the evaluation wrappers helps to see that the random forest learner has a higher RMSE for predicting the outcome $E[\Delta Y|D=1,X]$. Both models predict individuals' treatment (group) status with an accuracy of $1$.

In [ ]:
# Run estimation with evaluation: Linear model
set.seed(1234)
example_attgt_dml_eval_linear <- att_gt(yname = "Y",
                      tname = "period",
                      idname = "id",
                      gname = "G",
                      xformla = ~X,
                      data = dta,
                      est_method = doubleml_did_eval_linear,
                      print_details = TRUE)


summary(example_attgt_dml_eval_linear)


In [ ]:
# Run estimation with evaluation: Linear model
set.seed(1234)
example_attgt_dml_eval_rf <- att_gt(yname = "Y",
                      tname = "period",
                      idname = "id",
                      gname = "G",
                      xformla = ~X,
                      data = dta,
                      est_method = doubleml_did_eval_rf,
                      print_details = TRUE)


summary(example_attgt_dml_eval_rf)